In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("prices_with_timezone.csv")
print(df.head())
print(df.dtypes)

   Unnamed: 0                    date                          station_uuid  \
0        6953  2019-10-01 05:17:04+02  16f07bfd-0bde-4126-a393-ea8a7d053283   
1       34120  2019-10-01 08:09:04+02  16f07bfd-0bde-4126-a393-ea8a7d053283   
2       40389  2019-10-01 08:29:04+02  16f07bfd-0bde-4126-a393-ea8a7d053283   
3       74150  2019-10-01 10:25:05+02  16f07bfd-0bde-4126-a393-ea8a7d053283   
4       84881  2019-10-01 11:05:05+02  16f07bfd-0bde-4126-a393-ea8a7d053283   

   diesel     e5    e10  dieselchange  e5change  e10change  
0   1.349  1.519  1.499             1         1          1  
1   1.349  1.469  1.449             0         1          1  
2   1.309  1.449  1.429             1         1          1  
3   1.359  1.499  1.479             1         1          1  
4   1.319  1.449  1.429             1         1          1  
Unnamed: 0        int64
date             object
station_uuid     object
diesel          float64
e5              float64
e10             float64
dieselchange   

In [3]:
# Timezone conversion from +02:00 to UTC (+00:00)
df["date"] = pd.to_datetime(df["date"], infer_datetime_format=True, utc=True)
#test = pd.to_datetime(df["date"], format='%Y-%m-%d %H:%M:%S%z')
#test = df["date"].apply(lambda x: (dt.datetime.strptime(x + "00",'%Y-%m-%d %H:%M:%S%z')))
print(df.head())
print(df.dtypes)

   Unnamed: 0                      date                          station_uuid  \
0        6953 2019-10-01 03:17:04+00:00  16f07bfd-0bde-4126-a393-ea8a7d053283   
1       34120 2019-10-01 06:09:04+00:00  16f07bfd-0bde-4126-a393-ea8a7d053283   
2       40389 2019-10-01 06:29:04+00:00  16f07bfd-0bde-4126-a393-ea8a7d053283   
3       74150 2019-10-01 08:25:05+00:00  16f07bfd-0bde-4126-a393-ea8a7d053283   
4       84881 2019-10-01 09:05:05+00:00  16f07bfd-0bde-4126-a393-ea8a7d053283   

   diesel     e5    e10  dieselchange  e5change  e10change  
0   1.349  1.519  1.499             1         1          1  
1   1.349  1.469  1.449             0         1          1  
2   1.309  1.449  1.429             1         1          1  
3   1.359  1.499  1.479             1         1          1  
4   1.319  1.449  1.429             1         1          1  
Unnamed: 0                    int64
date            datetime64[ns, UTC]
station_uuid                 object
diesel                      float64
e5 

In [4]:
import datetime
from dateutil import tz
import numpy as np
timedelta = df["date"] - datetime.datetime(1970, 1, 1, tzinfo=tz.gettz('UTC'))
print(timedelta.head())
timedelta = timedelta / np.timedelta64(1, 's')
print(timedelta.head())
timedelta = timedelta.astype('int64')
print(timedelta.head())

0   18170 days 03:17:04
1   18170 days 06:09:04
2   18170 days 06:29:04
3   18170 days 08:25:05
4   18170 days 09:05:05
Name: date, dtype: timedelta64[ns]
0    1.569900e+09
1    1.569910e+09
2    1.569911e+09
3    1.569918e+09
4    1.569921e+09
Name: date, dtype: float64
0    1569899824
1    1569910144
2    1569911344
3    1569918305
4    1569920705
Name: date, dtype: int64


In [47]:
df_in = pd.read_csv("prices_with_timezone.csv")

In [48]:
# convert dataframe column date to seconds since epoch
def convert_df_to_epoch_seconds_utc(df: pd.DataFrame, column: str):
    df[column] = pd.to_datetime(df[column], infer_datetime_format=True, utc=True)
    timedelta = df[column] - datetime.datetime(1970, 1, 1, tzinfo=tz.gettz('UTC'))
    timedelta = timedelta / np.timedelta64(1, 's')
    timedelta = timedelta.astype('int64')
    df[column] = timedelta
    return df

In [49]:
df_utc_epoch = convert_df_to_epoch_seconds_utc(df_in, "date").head()
print(df_utc_epoch.head())

   Unnamed: 0        date                          station_uuid  diesel  \
0        6953  1569899824  16f07bfd-0bde-4126-a393-ea8a7d053283   1.349   
1       34120  1569910144  16f07bfd-0bde-4126-a393-ea8a7d053283   1.349   
2       40389  1569911344  16f07bfd-0bde-4126-a393-ea8a7d053283   1.309   
3       74150  1569918305  16f07bfd-0bde-4126-a393-ea8a7d053283   1.359   
4       84881  1569920705  16f07bfd-0bde-4126-a393-ea8a7d053283   1.319   

      e5    e10  dieselchange  e5change  e10change  
0  1.519  1.499             1         1          1  
1  1.469  1.449             0         1          1  
2  1.449  1.429             1         1          1  
3  1.499  1.479             1         1          1  
4  1.449  1.429             1         1          1  


In [50]:
def convert_df_from_epoch_to_iso8601_tz(df: pd.DataFrame, column: str, timezone: tz.tzfile = tz.gettz('UTC')):
    df[column] = pd.to_datetime(df[column], unit='s', utc=True)
    df[column] = df[column].dt.tz_convert(timezone)
    return df

In [51]:
print(convert_df_from_epoch_to_iso8601_tz(df_utc_epoch, "date", tz.gettz("Europe/Berlin")).head())
print(convert_df_from_epoch_to_iso8601_tz(df_utc_epoch, "date", tz.gettz("Pacific/Kiritimati")).head())

   Unnamed: 0                      date                          station_uuid  \
0        6953 2019-10-01 05:17:04+02:00  16f07bfd-0bde-4126-a393-ea8a7d053283   
1       34120 2019-10-01 08:09:04+02:00  16f07bfd-0bde-4126-a393-ea8a7d053283   
2       40389 2019-10-01 08:29:04+02:00  16f07bfd-0bde-4126-a393-ea8a7d053283   
3       74150 2019-10-01 10:25:05+02:00  16f07bfd-0bde-4126-a393-ea8a7d053283   
4       84881 2019-10-01 11:05:05+02:00  16f07bfd-0bde-4126-a393-ea8a7d053283   

   diesel     e5    e10  dieselchange  e5change  e10change  
0   1.349  1.519  1.499             1         1          1  
1   1.349  1.469  1.449             0         1          1  
2   1.309  1.449  1.429             1         1          1  
3   1.359  1.499  1.479             1         1          1  
4   1.319  1.449  1.429             1         1          1  
   Unnamed: 0                      date                          station_uuid  \
0        6953 2019-10-01 17:17:04+14:00  16f07bfd-0bde-4126-a393-e

C:\Users\steffen.lang\AppData\Local\Temp\ipykernel_24156\4089532118.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = pd.to_datetime(df[column], unit='s', utc=True)
C:\Users\steffen.lang\AppData\Local\Temp\ipykernel_24156\4089532118.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].dt.tz_convert(timezone)
C:\Users\steffen.lang\AppData\Local\Temp\ipykernel_24156\4089532118.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame